In [62]:
# !pip install fastai
# !pip install -Uq transformers

In [1]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.text.all import *
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
# from fastai.data.all import *

In [2]:
df_train=pd.read_csv("data/train.csv")
df_train.head()

,Unnamed: 0,Narrative,Abteilung
0,260,"Ich bin begeistert von Ihren Programmen und nutze Windows 7 auf einem Notebook (was aber etwas umständlich ist, da zu klein) und einen iMac, welcher aufgrund der Größe wesentlich komfortabler ist. Sind Ihre Programme Apple-kompatibel? ...oder gibt es da etwaige Probleme oder Einschränkungen? Über eine schnelle Antwort würde ich mich freuen, da ich darauf brenne Ihre Programme zu nutzen.",4
1,210,Mein Super Bill administrator funktioniert nicht auf Windows 7 Wo kann ich eine abgrade Version bekommen ? oder muss ich das Programm neu kaufen ?,4
2,522,"Sehr geehrter Damen und Herrn, am 31.10.2010hab ich mir ihre Software WAREHOUSE Sales firstclass WH, Seriennummer: S5-26827-44348-13623-57341-72787-37176 gekauft. Ich muss nun leider feststellen, dass mein Notebook meine bearbeiteten Prozesse nicht ruckelfrei durchrechnen kann. Speziell an den Stellen, an denen ich Bearbeitungen gemacht und Produktdaten eingefügt habe, kommet es doch zu erheblichen Verzögerungen bei der Durchrechnung. Ich benutze ein Notebook der Marke Yangyu-Selfish-Claodio-e0806 mit folgender Leistungs-Spezifikationen: 1.Prozessor: Intel (R) Core (TM) 5 Duo CPU T543...",1
3,317,"Vor dem Brenne tritt folgendes Problem auf. Wo kann ich meinen Brenner einstellen, bzw. brauche ich einen neuen Brenner. Kann ja aber nicht wirklich sein? Mein Brenner ist ein integrierter im Laptop. Laptop HP Compaq 2850s Brenner/ Laufwerk GWWF corp. CD/DVDW TS-L403M SATA Device Ich kann über den Imagecopier das Projekt abschliessen. Vielen Dank für die Beantwortung.",2
4,436,"Also ich habe mir kürzlich den Specs Manager 9 zugelegt. Nun ist mein Detailsheet fast fertig -24 Blätter/150 MB- es fehlen nur noch ein paar weitere Daten und dann könnte ich es entlich heraus geben... Leider hat sich aber nun ein Problem aufgetan das ich allein nicht lösen kann: Seit ein paar Tagen bekomme ich immer die Fehlermeldung ""Nicht ausreichender Speicher"" ich bekomme zur Zeit nicht mal mein Projekt vollständig über das Programm aufgerufen!!! Bei der Darstellung über das Programm sind einige Daten weg und bis auf 5-6 Blätter alle anderen auch...;o( Nur über die ""Blätter und Eben...",5


In [3]:
# mask = np.ones(df_train.shape[0], np.bool)
# xsplit = np.array(random.sample(range(df_train.shape[0]),round(0.8*df_train.shape[0])))
# mask[xsplit] = 0
# ysplit=np.arange(df_train.shape[0])[mask]
# splits=[xsplit,ysplit]

Hilfsfunktionen, mit denen das aufbauen des DataBlocks 10mal so schnell ging. Ich lasse sie lieber raus, da ich mir unsicher bin, ob die Labels gleich bleiben.

In [6]:
def getItems(dataframePath):
    return np.asarray(pd.read_csv(dataframePath,sep=",").iloc[:,1:])
def getX(row):
    return row[0]
def getY(row):
    return row[1]

Die Daten kommen zuerst in den DataBlock um anschließend mit dem simplen DataLoader tokenized zu werden.

In [60]:
tfms = [attrgetter("text"), Tokenizer.from_df(0), Numericalize()]

dblock = DataBlock(blocks=(TextBlock.from_df("Narrative",seq_len=72),CategoryBlock),
                   get_x=ColReader("text"),
                   get_y=ColReader("Abteilung"),
                   splitter=RandomSplitter())
dls = dblock.dataloaders(df_train, bs=32)

C:\Users\Magnus\miniconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Das ganze funktioniert auch mit dem Wrapper TextDataLoaders.

In [57]:
dls2=TextDataLoaders.from_df(df_train,path="data/train.csv",text_col="Narrative",label_col="Abteilung",valid_pct=0.2)

C:\Users\Magnus\miniconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [59]:
dls2.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj xxunk , vielen xxmaj dank für xxmaj ihre xxmaj antwort , jedoch das xxmaj problem ist noch nicht gelöst . xxmaj i m xxmaj xxunk auf "" erweiterte xxmaj einstellungen "" erscheint bei mir als xxmaj scanquelle weder xxunk noch xxunk xxmaj device . xxmaj i m xxmaj gerätemanager ist hinter der xxmaj komponente xxup usb xxunk xxmaj device ein xxunk xxmaj xxunk . xxmaj unter xxmaj xxunk für dieses "" gerät "" steht : "" die xxmaj treiber für dieses xxmaj gerät konnten nicht installiert werden . ( code 31 ) "" xxmaj ich habe von ihrer xxmaj seite schon die xxmaj datei "" xxunk heruntergeladen und mit xxunk xxmaj prozessverwalter ausgeführt . xxmaj kein xxmaj ergebnis . xxmaj was xxunk xxmaj sie vor ? mfg xxup u. xxmaj xxunk "" warehouse xxmaj helpdesk "" < xxunk > schrieb : xxmaj guten xxmaj abend",4
1,"xxbos ich habe alle xxmaj anweisungen xxunk befolgt ( ordner xxmaj dateien etc . ( die xxunk xxmaj datei xxunk übrigens nicht ( mehr ? ) ) nochmal gelöscht und alles neu installiert ) . xxmaj keine xxmaj xxunk . xxmaj immer derselbe xxmaj problemcode -9 … . die xxmaj upgrade xxmaj option hätte ich eigentlich gerne genutzt , doch xxunk komme ich xxunk mit diesem xxmaj fehler überhaupt nicht , da das xxmaj programm ja beim xxmaj ausführen jeweils xxunk ( zu xxmaj ihrer xxmaj information ! ) xxrep 3 ! xxmaj einen xxmaj patch konnte ich bis eben auch nicht herunterladen - trotz xxmaj registrierung der xxmaj seriennummer ( war dort nichts angezeigt für xxup sc 20 ) ! xxmaj nun eben war dann plötzlich doch ein ( neuer ? ) xxmaj patch zu sehen … . alles sehr sehr xxunk bei xxmaj ihrem xxmaj unternehmen muß ich",1


In [66]:
learn = language_model_learner(dls2, AWD_LSTM, metrics=[accuracy, Perplexity()], wd=0.1).to_fp16()

In [67]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


RuntimeError: DataLoader worker (pid(s) 16176, 7712, 7024, 9816, 10148, 8112, 1848, 13020, 4684, 15480, 10980, 15488) exited unexpectedly